# Remove Instrument Response

This is an example notebook for removing the instrument response to calibrate the data into physical units.

All the filters are represented as `mt_metadata.timeseries.filters` objects.  More information about these can be found in [mt_metadata filters](https://mt-metadata.readthedocs.io/en/latest/source/notebooks/filters_example.html).

This example will read in an existing MTH5 that has one station `CAS04`, one run `a`, and 2 channels `ex` and `hx`. 

In [11]:
from mth5.mth5 import MTH5

## Open the MTH5 file

This has 2 stations in it CAS04 and NVR08 both MTArray/Earthscope stations.  Each only has one run.  

In [12]:
m = MTH5()
m.open_mth5(r"c:\Users\jpeacock\Documents\GitHub\mth5\docs\examples\notebooks\ZU_CAS04_NVR08_small.h5")

Have a look at the channel summary.  This is a table of all the channels available in the MTH5 file.

In [13]:
ch_df = m.channel_summary.to_dataframe()
ch_df

,survey,station,run,latitude,longitude,elevation,component,start,end,n_samples,sample_rate,measurement_type,azimuth,tilt,units,hdf5_reference,run_hdf5_reference,station_hdf5_reference
0,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,ex,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,electric,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
1,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,hx,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,magnetic,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>


## Get EX channel

Here we will get an EX channel, it will return a `channel_dataset` object.

In [14]:
ex = m.get_channel("CAS04", "a", "ex", survey="CONUS_South")

In [15]:
ex

Channel Electric:
-------------------
	component:        ex
	data type:        electric
	data format:      int32
	data shape:       (11267,)
	start:            2020-06-02T19:00:00+00:00
	end:              2020-06-02T22:07:46+00:00
	sample rate:      1.0

### ChannelTS object

Convert the `ex` channel to a `ChannelTS` object which is based on an `xarray.DataArray` and has methods to working with the data.  More information can be found [Time Series Objects](https://mth5.readthedocs.io/en/latest/source/ts.html)

In [16]:
exts = ex.to_channel_ts()

In [17]:
exts.channel_response_filter

Filters Included:
pole_zero_filter:
	calibration_date = 1980-01-01
	comments = butterworth filter
	gain = 1.0
	name = electric field 5 pole butterworth low-pass
	normalization_factor = 313383.601119193
	poles = [ -3.883009+11.951875j  -3.883009-11.951875j -10.166194 +7.386513j
 -10.166194 -7.386513j -12.566371 +0.j      ]
	type = zpk
	units_in = mV/km
	units_out = mV/km
	zeros = []
--------------------
pole_zero_filter:
	calibration_date = 1980-01-01
	comments = butterworth filter
	gain = 1.0
	name = electric field 1 pole butterworth high-pass
	normalization_factor = 1.00000351809047
	poles = [-0.000167+0.j]
	type = zpk
	units_in = mV/km
	units_out = mV/km
	zeros = [ 0.+0.j]
--------------------
coefficient_filter:
	calibration_date = 1980-01-01
	comments = unit conversion
	gain = 1e-06
	name = mv per km to v per m
	type = coefficient
	units_in = mV/km
	units_out = V/m
--------------------
coefficient_filter:
	calibration_date = 1980-01-01
	comments = electric dipole
	gain = 94.0
	name

In [18]:
ex.channel_response_filter

Filters Included:
pole_zero_filter:
	calibration_date = 1980-01-01
	comments = butterworth filter
	gain = 1.0
	name = electric field 5 pole butterworth low-pass
	normalization_factor = 313383.601119193
	poles = [ -3.883009+11.951875j  -3.883009-11.951875j -10.166194 +7.386513j
 -10.166194 -7.386513j -12.566371 +0.j      ]
	type = zpk
	units_in = mV/km
	units_out = mV/km
	zeros = []
--------------------
pole_zero_filter:
	calibration_date = 1980-01-01
	comments = butterworth filter
	gain = 1.0
	name = electric field 1 pole butterworth high-pass
	normalization_factor = 1.00000351809047
	poles = [-0.000167+0.j]
	type = zpk
	units_in = mV/km
	units_out = mV/km
	zeros = [ 0.+0.j]
--------------------
coefficient_filter:
	calibration_date = 1980-01-01
	comments = unit conversion
	gain = 1e-06
	name = mv per km to v per m
	type = coefficient
	units_in = mV/km
	units_out = V/m
--------------------
coefficient_filter:
	calibration_date = 1980-01-01
	comments = electric dipole
	gain = 94.0
	name

In [19]:
m

/:
    |- Group: Experiment
    --------------------
        |- Group: Reports
        -----------------
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Surveys
        -----------------
            |- Group: CONUS_South
            ---------------------
                |- Group: Filters
                -----------------
                    |- Group: coefficient
                    ---------------------
                        |- Group: mv per km to v per m
                        ------------------------------
                        |- Group: v per m to v
                        ----------------------
                        |- Group: v to counts (electric)
                        --------------------------------
                        |- Group: v to counts (magnetic)
                        --------------------------------
                    |- Group: fap
                    -------------

In [20]:
m.close_mth5()

2022-04-05 16:34:39,255 [line 726] mth5.mth5.MTH5.close_mth5 - INFO: Flushing and closing c:\Users\jpeacock\Documents\GitHub\mth5\docs\examples\notebooks\ZU_CAS04_NVR08_small.h5
